In [1]:

import json
from pathlib import Path 
import numpy as np
import tifffile as tifff
import os
import os
import json
import numpy as np
from PIL import Image, ImageDraw
#from skimage.draw import polygon2mask

In [2]:
def polygon2mask(xysize, polygon_outline):
    mask = Image.new('L', xysize, 0)
    ImageDraw.Draw(mask).polygon(polygon_outline, outline=1, fill=1)
    return np.array(mask)
xysize=(1000,1000)

In [3]:
xysize=(1000,1000)

In [4]:
base_path = '/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/github/myeloma_standal/bone_masks_qupath'

In [5]:
for geofile in os.listdir(base_path):
    sample = geofile.split('.')[0]
    if geofile.endswith('.geojson'):
        with open(os.path.join(base_path, geofile)) as f:
            data = json.load(f)

        mask_combined = np.zeros((xysize[1], xysize[0]), dtype=np.uint8)

        for feature in data['features']:
            if feature['geometry']['type'] == 'Polygon':
                polygon_coords = feature['geometry']['coordinates'][0]
                polygon_outline = [(coord[0], coord[1]) for coord in polygon_coords]
                mask = polygon2mask(xysize, polygon_outline)
                mask_combined = np.logical_or(mask_combined, mask)
            elif feature['geometry']['type'] == 'MultiPolygon':
                for polygon in feature['geometry']['coordinates']:
                    polygon_outline = [(coord[0], coord[1]) for coord in polygon[0]]
                    mask = polygon2mask(xysize, polygon_outline)
                    mask_combined = np.logical_or(mask_combined, mask)

        mask_combined = (mask_combined * 255).astype(np.uint8)
        output_file = os.path.join('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/QC/bone_masks_qupath', sample + '_bone_masks.tiff')
        Image.fromarray(mask_combined).save(output_file)